In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
#设置文件目录
#训练集
trainDir ='./save/'
trainFakeDir = './save/fake/'
trainRealDir = './save/real/'
#测试集
valDir = './test/'
valFakeDir = './test/fake/'
valRealDir = './test/real/'

#创建模型
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(224, 224, 3)))
#卷积层，输出空间的维数为32,也可以说是输出特征图的深度为32，提取信息的窗口大小(3,3),卷积核的大小也为(3,3)
model.add(layers.MaxPooling2D((2, 2)))
#池化层，窗口大小为(2,2)，缩小特征图的尺寸
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
#扁平层,将多维的输入转化为一维的输出
model.add(layers.Dense(512, activation='relu'))
#全连接层，将提取的特征组合，得出结果
model.add(layers.Dense(1, activation='sigmoid'))

#设置损失函数，优化器，模型在训练和测试时的性能指标
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
  optimizer=optimizers.RMSprop(lr=1e-4),
  metrics=['acc'])

#配置图片生成器
from keras.preprocessing.image import ImageDataGenerator
#将图片像素缩小为[0,1]之间的浮点数
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#创建图片生成器
train_generator = train_datagen.flow_from_directory(
 trainDir,#图片地址
 target_size=(224, 224),
 batch_size=16,#设置批量数据的大小
 class_mode='binary')#设置返回标签的类型
#val_generator = test_datagen.flow_from_directory(
# valDir,
# target_size=(224, 224),
# batch_size=16,
# class_mode='binary')


#拟合模型
history = model.fit_generator(
  train_generator,
  steps_per_epoch=274,#迭代进入下一轮次需要抽取的批次
  epochs=20,#数据迭代的轮数
  #validation_data=val_generator,
 # validation_steps=8)#验证集用于评估的批次
)


In [ ]:
#保存模型
model.save('opticalflow.h5')

In [ ]:
#画出结果
import matplotlib.pyplot as plt

#查看变量，发现history.history中就只有这四个值，分别是准确度，验证集准确度，损失，验证集损失
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

#画两个图，分别是正确率和验证损失率
#正确率
plt.figure(1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()
#损失
plt.figure(2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()